In [ ]:
# Zelle 1 – Google Drive mounten (nur in Colab nötig)
# Damit später das finegetunte Modell und die Ergebnisse direkt in deinem Drive liegen.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# Installiere bitsandbytes für 4-Bit / 8-Bit Support
!pip install -q bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.0 MB/s eta 0:00:00


In [ ]:
# Zelle 2 – Bibliotheken importieren
# os und json für Datei-Operationen
# pandas zum Erstellen und Speichern von DataFrames
# transformers & peft zum Nachladen deines Modells/Adapters und Pipelines
import os
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from peft         import PeftModel

In [ ]:
# Zelle 3 – Prompt-Instruction definieren

INSTRUCTION = (
    "### Instruction:\n"
    "From the input text, extract the following fields ONLY.\n"
    "Format your response as newline-separated entries:\n\n"
    "category: value\n\n"
    "Do NOT include any additional text, headings, or explanations.\n\n"
    "Fields to extract:\n"
    "- risk_communication\n"
    "- single_case_base\n"
    "- absolute_risk_base\n"
    "- absolute_risk_new\n"
    "- absolute_number_base\n"
    "- absolute_number_new\n"
    "- absolute_risk_difference\n"
    "- relative_risk\n"
    "- absolute_number_difference\n"
    "- verbal_descriptor_base\n"
    "- verbal_descriptor_new\n"
    "- verbal_descriptor_change\n"
    "- population_size\n"
    "- reference_class_description_base\n"
    "- reference_class_description_new\n"
    "- reference_class_size_base\n"
    "- reference_class_size_new\n"
    "- source_base\n"
    "- source_new\n"
    "- topic_and_unit\n\n"
)

In [ ]:
# Zelle 4 - Hugging Face Connection
from huggingface_hub import login
login()
#

In [ ]:
# Zelle 5 – Basis-Modell & QLoRA-Adapter laden

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

base = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto"
)
# Schalte Training-Features ab und aktiviere den Cache für schnelle Inference
base.eval()
base.gradient_checkpointing_disable()
base.config.use_cache = True

# 2) Lade den LoRA-Adapter (deine feingetunten Gewichte) aus deinem Drive-Ordner
ADAPTER_PATH = "/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter-1306a"
model = PeftModel.from_pretrained(base, ADAPTER_PATH)
model.eval()  # Setze das Modell in den Inference-Modus


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj

In [ ]:
# Zelle 6 – Tokenizer laden -> Modell und Tokenizer gehören zusammen (Tokenizer wurden vielleicht auch geändert)

# Der Tokenizer liegt im gleichen Adapter-Ordner, damit die Token-IDs übereinstimmen
tokenizer = AutoTokenizer.from_pretrained(
    ADAPTER_PATH,
    trust_remote_code=True
)
# Falls kein pad_token definiert ist, setze ihn auf den eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Zelle 7 - Pipeline einrichten

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=300, # tendenziell hochschrauben? -> oder kommt dann explanation part dazu?
    do_sample=False,        # Greedy = deterministisch
    return_full_text=True, # nur generierten Text
    use_cache=True,
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Zelle 8 – Interaktives Prompting in der Konsole
while True:
    user_input = input("\n✍️ Gib einen Text ein (oder 'quit' zum Beenden): ")
    if user_input.lower() == "quit":
        print("👋 Sitzung beendet.")
        break

    prompt = (
        INSTRUCTION
        + f"### Input:\n{user_input.strip()}\n\n"
        + "### Output:\n"
    )
    gen = pipe(prompt)[0]["generated_text"]
    print(gen)
    # alles ab ### Input: behalten (inkl. Marker)
    out = "### Input:" + gen.split("### Input:")[-1].strip()
    #print("\n🔍 Modell-Antwort:\n", out)



✍️ Gib einen Text ein (oder 'quit' zum Beenden): In 2025, approximately 0.8% of Canadians are projected to be living with  inflammatory bowel disease (IBD), a figure expected to climb to 1% by the 2030s and 2040s.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Instruction:
From the input text, extract the following fields ONLY.
Format your response as newline-separated entries:

category: value

Do NOT include any additional text, headings, or explanations.

Fields to extract:
- risk_communication
- single_case_base
- absolute_risk_base
- absolute_risk_new
- absolute_number_base
- absolute_number_new
- absolute_risk_difference
- relative_risk
- absolute_number_difference
- verbal_descriptor_base
- verbal_descriptor_new
- verbal_descriptor_change
- population_size
- reference_class_description_base
- reference_class_description_new
- reference_class_size_base
- reference_class_size_new
- source_base
- source_new
- topic_and_unit

### Input:
In 2025, approximately 0.8% of Canadians are projected to be living with  inflammatory bowel disease (IBD), a figure expected to climb to 1% by the 2030s and 2040s.

### Output:
risk_communication: 1.0
single_case_base: 0.0
absolute_risk_base: 0.008
absolute_risk_new: 0.01
absolute_number_base: null
ab

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Instruction:
From the input text, extract the following fields ONLY.
Format your response as newline-separated entries:

category: value

Do NOT include any additional text, headings, or explanations.

Fields to extract:
- risk_communication
- single_case_base
- absolute_risk_base
- absolute_risk_new
- absolute_number_base
- absolute_number_new
- absolute_risk_difference
- relative_risk
- absolute_number_difference
- verbal_descriptor_base
- verbal_descriptor_new
- verbal_descriptor_change
- population_size
- reference_class_description_base
- reference_class_description_new
- reference_class_size_base
- reference_class_size_new
- source_base
- source_new
- topic_and_unit

### Input:
Living in a city with high air pollution increases your risk of lung disease by 10%. While that may seem minor, it translates to an increase from 30 in 1,000 to 33 in 1,000 people developing chronic respiratory issues over a decade.

### Output:
risk_communication: 1.0
single_case_base: 0.0
absolute_ri

...

### Instruction:
From the input text, extract the following fields ONLY.
Format your response as newline-separated entries:

category: value

Do NOT include any additional text, headings, or explanations.

Fields to extract:
- risk_communication
- single_case_base
- absolute_risk_base
- absolute_risk_new
- absolute_number_base
- absolute_number_new
- absolute_risk_difference
- relative_risk
- absolute_number_difference
- verbal_descriptor_base
- verbal_descriptor_new
- verbal_descriptor_change
- population_size
- reference_class_description_base
- reference_class_description_new
- reference_class_size_base
- reference_class_size_new
- source_base
- source_new
- topic_and_unit

### Input:
The risk of getting shot by police for a black person is 10% during his lifetime. For a white person, it is 50% lower.

### Output:
risk_communication: 1.0
single_case_base: 0.0
absolute_risk_base: 0.1
absolute_risk_new: null
absolute_number_base: null
absolute_number_new: null
absolute_risk_difference

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Instruction:
From the input text, extract the following fields ONLY.
Format your response as newline-separated entries:

category: value

Do NOT include any additional text, headings, or explanations.

Fields to extract:
- risk_communication
- single_case_base
- absolute_risk_base
- absolute_risk_new
- absolute_number_base
- absolute_number_new
- absolute_risk_difference
- relative_risk
- absolute_number_difference
- verbal_descriptor_base
- verbal_descriptor_new
- verbal_descriptor_change
- population_size
- reference_class_description_base
- reference_class_description_new
- reference_class_size_base
- reference_class_size_new
- source_base
- source_new
- topic_and_unit

### Input:
If you buy this lottery ticket, your chances of becoming a millionaire are 10 times higher

### Output:
risk_communication: 1.0
single_case_base: 1.0
absolute_risk_base: null
absolute_risk_new: null
absolute_number_base: null
absolute_number_new: null
absolute_risk_difference: null
relative_risk: 10.0


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Instruction:
From the input text, extract the following fields ONLY.
Format your response as newline-separated entries:

category: value

Do NOT include any additional text, headings, or explanations.

Fields to extract:
- risk_communication
- single_case_base
- absolute_risk_base
- absolute_risk_new
- absolute_number_base
- absolute_number_new
- absolute_risk_difference
- relative_risk
- absolute_number_difference
- verbal_descriptor_base
- verbal_descriptor_new
- verbal_descriptor_change
- population_size
- reference_class_description_base
- reference_class_description_new
- reference_class_size_base
- reference_class_size_new
- source_base
- source_new
- topic_and_unit

### Input:
If you buy this lottery ticket, your chances of becoming a millionaire are 100% higher.

### Output:
risk_communication: 1.0
single_case_base: 0.0
absolute_risk_base: null
absolute_risk_new: null
absolute_number_base: null
absolute_number_new: null
absolute_risk_difference: null
relative_risk: 1.0
abso